# 아이유팬이 좋아할 만한 다른 아티스트 찾기

## 데이터셋

2) 분석해 봅시다.  
ratings에 있는 유니크한 영화 개수  
ratings에 있는 유니크한 사용자 수  
가장 인기 있는 영화 30개(인기순)  
3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.  
4) CSR matrix를 직접 만들어 봅시다.  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.  
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [13]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [14]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')


original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [15]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [16]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [17]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [18]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [19]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [20]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [21]:
my_favorite = ['Jurassic Park (1993)' , 'Toy Story (1995)' ,'Matrix, The (1999)' ,'American Beauty (1999)' ,'Terminator 2: Judgment Day (1991)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1,5
476,6041,480,5
585,6041,589,5
2502,6041,2571,5
2789,6041,2858,5


In [22]:
favorite_movie_id

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2789,2858,American Beauty (1999),Comedy|Drama


In [23]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


## CSR matrix

In [24]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,1,5
476,6041,480,5
585,6041,589,5
2502,6041,2571,5


In [25]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [26]:
num_user

6040

In [27]:
num_movie

3628

In [28]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
0       6041
476     6041
585     6041
2502    6041
2789    6041
Name: user_id, Length: 836483, dtype: int64

In [29]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
0          1
476      480
585      589
2502    2571
2789    2858
Name: movie_id, Length: 836483, dtype: int64

In [30]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델 훈련

In [31]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [32]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)


In [33]:
csr_data_transpose = csr_data.T
csr_data_transpose


<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [34]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 선호하는 영화, 그 외 영화

In [40]:
my_vector, jurassic_vector = als_model.user_factors[6041], als_model.item_factors[480]


In [41]:
my_vector

array([ 2.6306435e-01, -2.0475981e-01,  1.6474444e-01,  1.0980814e-01,
        9.3544513e-01, -3.5639849e-02, -4.3893677e-01,  1.6828440e-01,
        3.2870990e-01, -4.6265313e-01, -5.2640710e-02, -2.2878632e-01,
       -4.9306922e-02, -5.5785286e-01, -3.3200800e-01, -1.8697698e-01,
        1.2341820e-01, -3.0504281e-03,  3.6692411e-01,  6.8102434e-02,
       -3.3726045e-01,  6.7046821e-01, -5.8985507e-01, -4.2850539e-02,
       -3.1047162e-01, -6.1634660e-01, -3.6253241e-01, -3.2382544e-02,
        5.5320543e-01, -1.8808877e-01, -7.0018798e-01,  8.6788875e-01,
        8.9371659e-02,  6.5017498e-01, -2.3683520e-01, -8.4888361e-02,
       -6.1340052e-01,  1.1220666e+00, -1.8551810e-01,  1.1422702e+00,
       -1.8183251e-01, -4.5454738e-01,  4.0153894e-01,  2.3329610e-01,
       -8.9946785e-04, -2.3925699e-02, -9.6219614e-02, -8.5908180e-01,
       -2.6598573e-01,  5.1579338e-01,  4.6580669e-02, -1.9323133e-01,
       -9.1627872e-01,  3.8183656e-01, -2.8322780e-01,  6.4668804e-01,
      

In [42]:
jurassic_vector

array([ 0.01944167,  0.0275675 ,  0.01394277,  0.01180042,  0.02942595,
        0.02491731, -0.00276717, -0.00546917,  0.00385999, -0.00882773,
       -0.02134841, -0.00212044, -0.00905109, -0.01503511, -0.0107117 ,
        0.02289857, -0.00417393,  0.01178737,  0.03143255, -0.02609786,
        0.02275052,  0.02515005, -0.04762876,  0.01099888,  0.00582067,
        0.00104951,  0.01449694,  0.00873649,  0.00597817,  0.00278517,
       -0.01208582,  0.02266173, -0.01417382,  0.01518506, -0.01405284,
        0.0246243 , -0.00629609,  0.04315902,  0.01327802,  0.02681045,
       -0.01273895,  0.00691429,  0.04340918,  0.01276966,  0.01789902,
        0.03381509,  0.018218  , -0.01863985, -0.00044223,  0.01695945,
       -0.00809077,  0.03181316, -0.03156799,  0.01726667, -0.01417379,
        0.02531127, -0.00199656,  0.04627046,  0.02099412, -0.00718358,
        0.04354066, -0.01658467,  0.05581382,  0.03536685,  0.00183526,
        0.03210701,  0.00633004, -0.02430176, -0.0018784 , -0.03

In [43]:
np.dot(my_vector, jurassic_vector)

0.70554644

In [44]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.50196975

### 비슷한 영화

In [45]:
favorite_movie = 'American Beauty (1999)'
movie_id = movies[movies['title']=='American Beauty (1999)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(2858, 1.0),
 (2997, 0.7281322),
 (2599, 0.44952092),
 (2396, 0.39931223),
 (1787, 0.34595206),
 (608, 0.34245214),
 (2028, 0.3381857),
 (110, 0.32174218),
 (2762, 0.31541857),
 (2706, 0.31500143),
 (1259, 0.30530688),
 (296, 0.30028027),
 (527, 0.2995635),
 (989, 0.2989114),
 (593, 0.29873663)]

In [46]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
293,296,Pulp Fiction (1994),Crime|Drama
523,527,Schindler's List (1993),Drama|War
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
604,608,Fargo (1996),Crime|Drama|Thriller
977,989,Schlafes Bruder (Brother of Sleep) (1995),Drama
1239,1259,Stand by Me (1986),Adventure|Comedy|Drama
1728,1787,Paralyzing Fear: The Story of Polio in America...,Documentary
1959,2028,Saving Private Ryan (1998),Action|Drama|War
2327,2396,Shakespeare in Love (1998),Comedy|Romance


### 좋아하는 영화

In [47]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1580, 0.5449036),
 (3114, 0.4467556),
 (110, 0.40987563),
 (2916, 0.39955676),
 (457, 0.3651894),
 (2355, 0.3299507),
 (2028, 0.32972133),
 (34, 0.32220656),
 (1210, 0.30932888),
 (2997, 0.2922292),
 (2396, 0.27259022),
 (260, 0.25055957),
 (1196, 0.24680969),
 (1240, 0.2448525),
 (356, 0.23909965),
 (2762, 0.23799235),
 (1610, 0.23512745),
 (551, 0.2282678),
 (1259, 0.22713926),
 (2599, 0.22620362)]

In [48]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
33,34,Babe (1995),Children's|Comedy|Drama
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
352,356,Forrest Gump (1994),Comedy|Romance|War
453,457,"Fugitive, The (1993)",Action|Thriller
547,551,"Nightmare Before Christmas, The (1993)",Children's|Comedy|Musical
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
1239,1259,Stand by Me (1986),Adventure|Comedy|Drama
